In [1]:
# STEINMETZ PREPROCESSING

In [ ]:
# Set up

# Import packages
import os, requests
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pandas as pd

# Download and load data 
from data.get_data import load,download

download()

# Load data
alldat = load()

Loading data. Patience, padawan ...


In [ ]:
# Basic information about Steinmetz dataset: descriptive for each session
session = []
mouse_name = []
number_trials = []
total_trials = 0
for i in range(len(alldat)):
    dat = alldat[i]
    session.append(i+1)
    mouse_name.append(dat.get('mouse_name'))
    number_trials.append(len(dat.get('gocue')))
    total_trials += len(dat.get('gocue'))
    
first_dict = {'Session': session, 'Mouse Name': mouse_name, 'Number of Trials': number_trials}
basic_data_info = pd.DataFrame(first_dict, columns=['Session', 'Mouse Name','Number of Trials'])

print(basic_data_info)
print('Total number of trials: ', total_trials)

In [ ]:
#DataFrame for EVERY-SINGLE-TRIAL *clap emoji* (excluding passive trials: just Go (left/right) or NoGo trials)

s = 0
test_sessions = [1, 4, 8, 12, 19, 22, 25, 30, 35]

# Initialize empty list 
session = [] 
session_type = []
mouse_name = []  
trial_number = []
go_trial = []   #either TRUE (Go trial) or FALSE(= NoGo trial)
trial_type = []  # A, B, C, D, or E
stim_loc = []
ideal_resp = []
gocue = [] #dat['gocue']: when the go cue sound was played. 
latency = []
response_time = []
mouse_resp = []
wheel_velocity = []
wheel_acceleration = []
pres_acc = []
feedback_onset = []
feedback_type = []
contrast_left = []
contrast_right = []
contrast_diff = []
pres_difficulty = []

# Loop to fill all lists with data from Steinmetz dataset
for dat in alldat:
    s += 1
    for t in range(len(dat.get('gocue'))):      #just because the length of this field = number of trials for a given session
        # session #
        session.append(s) #session number

        if session[t] == 1 or session[t] == 4 or session[t] == 8 or session[t] == 12 or session[t] == 19 or session[t] == 22 or session[t] == 25 or session[t] == 30 or session[t] == 35:
            session_type.append('test')
        else: 
            session_type.append('train') 
        
        # mouse name
        mouse_name.append(dat.get('mouse_name')) #you know... just the name of the little fellow
        
        # trial number
        trial_number.append(t+1)      #trial number for that particular session

        # whether it was a go trial or not
        go_trial.append((dat.get('contrast_left')[t] != 0) or (dat.get('contrast_right')[t] != 0))
        
        # trial type, stimulus location on pres trial, and ideal response
        if dat.get('contrast_left')[t] == 0 and dat.get('contrast_right')[t] != 0:
            trial_t = 'A'
            stim_l = -1
            ideal_r = -1
        elif dat.get('contrast_left')[t] != 0 and dat.get('contrast_right')[t] == 0:
            trial_t = 'B'
            stim_l = 1
            ideal_r = 1
        elif dat.get('contrast_left')[t] == dat.get('contrast_right')[t] and dat.get('contrast_left')[t] != 0:
            trial_t = 'C'
            stim_l = 2
            ideal_r = np.nan
        elif dat.get('contrast_right')[t] and dat.get('contrast_left')[t] != 0 and dat.get('contrast_right')[t] > dat.get('contrast_left')[t] :
            trial_t = 'D'
            stim_l = 2
            ideal_r = -1
        elif dat.get('contrast_right')[t] and dat.get('contrast_left')[t]  != 0 and dat.get('contrast_right')[t] < dat.get('contrast_left')[t] :
            trial_t = 'E'
            stim_l= 2
            ideal_r = 1
        elif dat.get('contrast_left')[t] == 0 and dat.get('contrast_right')[t] == 0:
            trial_t= 'F'
            stim_l = 0
            ideal_r = 0
        else:
            trial_t = np.nan
            stim_l = np.nan
            ideal_r = np.nan
            
        trial_type.append(trial_t) 
        stim_loc.append(stim_l)
        ideal_resp.append(ideal_r)
        
        # go cue onset
        gocue.append(dat.get('gocue')[t][0]*1000) #the sencond index value is just there so that we can directly access floats from the DF
        
        # mouse response time 
        response_time.append(dat.get('response_time')[t][0]*1000)
        
        # latency 
        latency.append(dat.get('response_time')[t][0]*1000 - dat.get('gocue')[t][0]*1000)

        # append mouse's response
        mouse_resp.append(dat.get('response')[t])
        
        # velocity
        wheel_velocity.append(dat['wheel'][0][0] * .135) 
        
        # wheel acceleration
        wheel_acceleration.append(np.diff(wheel_velocity[t]/10))
    
        # feedback onset 
        feedback_onset.append((dat.get('feedback_time')[t][0]*1000))
        
        # feedback type 
        feedback_type.append(dat.get('feedback_type')[t]) #feedback type: positive (+1) means reward, negative (-1) means white noise burst 
       
        # stimulus contrasts and differences between them
        contrast_left.append(dat.get('contrast_left')[t])
        contrast_right.append(dat.get('contrast_right')[t])
        
        if stim_loc[t] != 0:
            contrast_diff.append((dat.get('contrast_left')[t] - dat.get('contrast_right')[t]).astype(float))
        else:
            contrast_diff.append(np.nan)

        # present difficulty
        if (dat.get('contrast_left')[t] != 0) or (dat.get('contrast_right')[t] != 0):
            pres_difficulty.append(1-np.abs(dat.get('contrast_left')[t]-dat.get('contrast_right')[t]))
        else:
            pres_difficulty.append(np.nan)
            
        # mouse's accuracy on present trial
        if feedback_type[t] == 1: 
            mouse_acc_binary = 1
            pres_acc.append(mouse_acc_binary)
        else:
            mouse_acc_binary = 0
            pres_acc.append(mouse_acc_binary)


In [ ]:
dict = {'session': session, 
        'session_type': session_type,
        'mouse_name': mouse_name, 
        'trial_number': trial_number,
        'trial_type': trial_type,
        'go_trial': go_trial,
        'stim_loc': stim_loc,
        'ideal_resp': ideal_resp,
        'gocue': gocue,
        'response_time': response_time,
        'latency': latency,
        'mouse_resp': mouse_resp,
        'wheel_velocity': wheel_velocity,
        'wheel_acceleration': wheel_acceleration,
        'feedback_onset': feedback_onset,
        'feedback_type': feedback_type,
        'contrast_left': contrast_left,
        'contrast_right': contrast_right,
        'contrast_diff': contrast_diff,
        'pres_difficulty': pres_difficulty,
        'pres_acc': pres_acc,
       }

data_extracted = pd.DataFrame(dict)
data_extracted.head()

In [ ]:
# Get previous accuracy and previous difficulty 

# Get previous latency
empty = np.empty([len(data_extracted['mouse_resp']), 1])
empty[0] = np.nan #ignore first index because there is no previous trial to this index

for i in range(len(empty)-1):
    empty[i+1] = data_extracted['latency'][i]

prev_latency = pd.DataFrame(empty)
prev_latency = prev_latency.rename(columns={0: 'prev_latency'})

# Get future latency 
empty = np.empty([len(data_extracted['mouse_resp']), 1])
empty[-1] = np.nan #ignore first index because there is no previous trial to this index

for i in range(len(empty)-1):
    empty[i] = data_extracted['latency'][i]

fut_latency = pd.DataFrame(empty)
fut_latency = fut_latency.rename(columns={0: 'fut_latency'})

# Get previous difficulty
empty = np.empty([len(data_extracted['mouse_resp']), 1])
empty[0] = np.nan #ignore first index because there is no previous trial to this index

for i in range(len(empty)-1):
    empty[i+1] = data_extracted['pres_difficulty'][i]

prev_difficulty = pd.DataFrame(empty)
prev_difficulty = prev_difficulty.rename(columns={0: 'prev_difficulty'})

# Get future difficulty
empty = np.empty([len(data_extracted['mouse_resp']), 1])
empty[-1] = np.nan #ignore first index because there is no previous trial to this index

for i in range(len(empty)-1):
    empty[i] = data_extracted['pres_difficulty'][i]

fut_difficulty = pd.DataFrame(empty)
fut_difficulty = fut_difficulty.rename(columns={0: 'fut_difficulty'})

# Get previous accuracy
empty = np.empty([len(data_extracted['mouse_resp']), 1])
empty[0] = np.nan #ignore first index because there is no previous trial to this index

for i in range(len(empty)-1):
    empty[i+1] = data_extracted['pres_acc'][i]

prev_acc = pd.DataFrame(empty)
prev_acc = prev_acc.rename(columns={0: 'prev_acc'})

# Get future accuracy
empty = np.empty([len(data_extracted['mouse_resp']), 1])
empty[-1] = np.nan #ignore first index because there is no previous trial to this index

for i in range(len(empty)-1):
    empty[i] = data_extracted['pres_acc'][i]

fut_acc = pd.DataFrame(empty)
fut_acc = fut_acc.rename(columns={0: 'fut_acc'})

# Combine into one df
prevs_futures = pd.concat([prev_latency, fut_latency, prev_difficulty, fut_difficulty, prev_acc, fut_acc], axis = 1)

prevs_futures.head()

In [ ]:
# Combine everything into one DF
data_extracted_unified = pd.concat([data_extracted, prevs_futures], axis = 1)

# Make sure all dfs are the same length (i.e. have the same number of rows)

if len(data_extracted) == len(prevs_futures) == len(data_extracted_unified):
    print('length of all dfs match! woo you are in the clear bb')

data_extracted_unified.columns

In [ ]:
# print dytpes held in each column of the df 

print('column: "session", dtype:', data_extracted_unified['session'].dtype)
print('column: "session_type", dtype:', data_extracted_unified['session_type'].dtype)
print('column: "mouse_name", dtype:', data_extracted_unified['mouse_name'].dtype)
print('column: "trial_number", dtype:', data_extracted_unified['trial_number'].dtype)
print('column: "trial_type", dtype:', data_extracted_unified['trial_type'].dtype)
print('column: "go_trial", dtype:', data_extracted_unified['go_trial'].dtype)
print('column: "stim_loc", dtype:', data_extracted_unified['stim_loc'].dtype)
print('column: "ideal_resp", dtype:', data_extracted_unified['ideal_resp'].dtype)
print('column: "gocue", dtype:', data_extracted_unified['gocue'].dtype)
print('column: "response_time", dtype:', data_extracted_unified['response_time'].dtype)
print('column: "mouse_resp", dtype:', data_extracted_unified['mouse_resp'].dtype)
print('column: "wheel_velocity", dtype:', data_extracted_unified['wheel_velocity'].dtype)
print('column: "wheel_acceleration", dtype:', data_extracted_unified['wheel_acceleration'].dtype)
print('column: "pres_acc", dtype:', data_extracted_unified['pres_acc'].dtype)
print('column: "feedback_onset", dtype:', data_extracted_unified['feedback_onset'].dtype)
print('column: "feedback_type", dtype:', data_extracted_unified['feedback_type'].dtype)
print('column: "contrast_left", dtype:', data_extracted_unified['contrast_left'].dtype)
print('column: "contrast_right", dtype:', data_extracted_unified['contrast_right'].dtype)
print('column: "contrast_diff", dtype:', data_extracted_unified['contrast_diff'].dtype)
print('column: "prev_latency", dtype:', data_extracted_unified['prev_latency'].dtype)
print('column: "fut_latency", dtype:', data_extracted_unified['fut_latency'].dtype)
print('column: "pres_difficulty", dtype:', data_extracted_unified['pres_difficulty'].dtype)
print('column: "prev_difficulty", dtype:', data_extracted_unified['prev_difficulty'].dtype)
print('column: "fut_difficulty", dtype:', data_extracted_unified['fut_difficulty'].dtype)
print('column: "pres_acc", dtype:', data_extracted_unified['pres_acc'].dtype)
print('column: "prev_acc", dtype:', data_extracted_unified['prev_acc'].dtype)
print('column: "fut_acc", dtype:', data_extracted_unified['fut_acc'].dtype)
